# Interactive data visualizations

Using python and the data catalog we can plot the data.

Required packages:
  1. *siphon* - for the TDS Catalog and remote data access 
  2. *netCDF4* for accessing encoded date information in the data file
  3. *dateutil* for date parsing
  4. *bokeh* for plotting

First, if running on colab, install the siphon package if missing

In [247]:
import os, sys, subprocess
if "google.colab" in sys.modules:
    try:
        from siphon.catalog import TDSCatalog
    except:
        cmd = "pip install siphon"
        process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        from siphon.catalog import TDSCatalog

Import required packages:

In [250]:
from pprint import pprint
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from datetime import datetime
from dateutil import parser
from netCDF4 import Dataset, num2date
from siphon.catalog import TDSCatalog

The output is on a notebook.

In [251]:
output_notebook()

Loading BokehJS ...

Useful lambda function:

In [252]:
dt_time = lambda x : parser.parse(str(x))

Access the data catalog on the server and retrieve the list of the dataset files:

In [253]:
catUrl = "http://140.105.45.139/thredds/catalog/testAll/catalog.xml";
catalog = TDSCatalog(catUrl)
filenames = list(catalog.datasets)
pprint(filenames)

['fluveswatersensors-0001_unesco-07_2022-02-16.nc',
 'fluveswatersensors-0001_unesco-07_2022-02-17.nc',
 'fluveswatersensors-0001_unesco-07_2022-02-18.nc',
 'fluveswatersensors-0001_unesco-07_2022-03-24.nc',
 'meteowindiotpro-0001_meteowind-iot-02_2022-02-16.nc',
 'meteowindiotpro-0001_meteowind-iot-02_2022-02-17.nc',
 'meteowindiotpro-0001_meteowind-iot-02_2022-02-18.nc',
 'meteowindiotpro-0001_meteowind-iot-02_2022-03-23.nc',
 'meteowindiotpro-0001_meteowind-iot-02_2022-03-24.nc',
 'weatherstation-0001_helix-06_2022-02-16.nc',
 'weatherstation-0001_helix-06_2022-02-17.nc',
 'weatherstation-0001_helix-06_2022-02-18.nc',
 'weatherstation-0001_helix-06_2022-03-23.nc',
 'weatherstation-0001_helix-06_2022-03-24.nc',
 'weatherstation-0001_helix-06_2022-03-25.nc',
 'weatherstation-0001_helix-07_2022-02-16.nc',
 'weatherstation-0001_helix-07_2022-02-17.nc',
 'weatherstation-0001_helix-07_2022-02-18.nc',
 'weatherstation-0001_helix-07_2022-03-23.nc',
 'weatherstation-0001_helix-07_2022-03-24.

Create the plot:

In [263]:
plt = figure(x_axis_type="datetime", width=800)

This function will open one of the files and return the handle and the list of plottable variables:

In [264]:
def access(filename):
    remote = catalog.datasets[filename]
    ds = Dataset(remote.access_urls['OpenDAP'])
    info = type('', (), {})()
    info.dev = ds.Device
    info.lat = f'{ds.Sensor_Latitude:.3f}'
    info.lon = f'{ds.Sensor_Longitude:.3f}'
    info.hgt = f'{ds.Sensor_Altitude:.1f}'
    time = ds.variables['time']
    info.date = [dt_time(x) for x in
           num2date(time[:],units=time.units,calendar=time.calendar)]    
    plottable = list(x for x in ds.variables if 'time' in ds.variables[x].dimensions)
    plottable = list(x for x in plottable if 'time' not in x)
    displaynames = list(ds.variables[x].long_name for x in plottable)
    selectable = dict(zip(displaynames,plottable))
    return (ds,info,selectable)

Let us now use our access method to retrieve the dataset and the list of plottable variables accessing the latest file of the list of the 'weatherstation' for the 'helix-09' station:

In [265]:
last_helix = list(x for x in filenames if 'weatherstation' in x and 'helix-07' in x)
ds, info, selectable = access(last_helix[-1])

Let us examine the list of plottable variables:

In [266]:
pprint(sorted(selectable))

['Maximum Total Solar Irradiance',
 'Maximum temperature',
 'Min time between rain gauge clicks',
 'Minimum temperature',
 'Pressure',
 'Rain',
 'Relative Humidity',
 'Temperature',
 'Total Solar Irradiance']


And the retrieved station informations:

In [267]:
print('Device = ', info.dev)
print('Latitude = ', info.lat)
print('Longitude = ', info.lon)
print('Elevation = ', info.hgt)

Device =  helix-07
Latitude =  45.703
Longitude =  13.721
Elevation =  80.0


Let us select the 'Temperature' variable and obtain the data:

In [268]:
var = ds.variables[selectable['Temperature']]
info.varname = var.long_name
info.varunit = var.units
info.data = var[:]

Here the plotter funtion:

In [269]:
def plotter(plt,info):
    plt.line(info.date,info.data,legend_label = info.varname)
    plt.title.text = ( info.dev + ' ' + info.varname + ' [' +
                       info.varunit + ']\n' +
                      '[lat: '+ info.lat + ', lon: ' + info.lon +
                       ', elv: ' + info.hgt + ']' )
    plt.xaxis.axis_label = 'Time'
    plt.yaxis.axis_label = info.varname + ' [' + info.varunit + ']'
    plt.axis.axis_label_text_font_style = "bold"
    return plt

And the plot:

In [272]:
plt = plotter(plt,info)
show(plt,notebook_handle=True)